In [23]:
from webdriver_manager.chrome import ChromeDriverManager
print(ChromeDriverManager(driver_version='120').install())

/bin/sh: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome: No such file or directory


/Users/mac/.wdm/drivers/chromedriver/mac64/120/chromedriver-mac-x64/chromedriver


/bin/sh: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome: No such file or directory


In [1]:
from selenium import webdriver
browser=webdriver.Chrome()

In [2]:
url='https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [3]:
#한개의 베스트 셀러 링크
from selenium.webdriver.common.by import By
browser.find_element(By.CLASS_NAME,'gd_name').get_attribute("href")
#1페이지 전체 링크
datas = browser.find_elements(By.CLASS_NAME,'gd_name')
for i in datas:
    print(i.get_attribute("href"))

https://www.yes24.com/Product/Goods/124999476
https://www.yes24.com/Product/Goods/126010792
https://www.yes24.com/Product/Goods/125992096
https://www.yes24.com/Product/Goods/125295101
https://www.yes24.com/Product/Goods/126029503
https://www.yes24.com/Product/Goods/125491624
https://www.yes24.com/Product/Goods/125651056
https://www.yes24.com/Product/Goods/125970283
https://www.yes24.com/Product/Goods/125576919
https://www.yes24.com/Product/Goods/117014613
https://www.yes24.com/Product/Goods/122120495
https://www.yes24.com/Product/Goods/123675187
https://www.yes24.com/Product/Goods/126029190
https://www.yes24.com/Product/Goods/125973831
https://www.yes24.com/Product/Goods/90926731
https://www.yes24.com/Product/Goods/96547408
https://www.yes24.com/Product/Goods/123400303
https://www.yes24.com/Product/Goods/124043812
https://www.yes24.com/Product/Goods/123930880
https://www.yes24.com/Product/Goods/125384202
https://www.yes24.com/Product/Goods/124926484
https://www.yes24.com/Product/Goods/

In [4]:
import time
link_list =[]
for i in range(1,4):
    url=f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    datas = browser.find_elements(By.CLASS_NAME,'gd_name')
    for i in datas:
        link_list.append(i.get_attribute("href"))
    time.sleep(3)
print(link_list)
print(len(link_list))

KeyboardInterrupt: 

In [ ]:
#for link in link_list:
    #상세 페이지 이동
#    browser.get(link)
browser.get(link_list[0])

title=browser.find_element(By.CLASS_NAME,'gd_name').text
author=browser.find_element(By.CLASS_NAME,'gd_auth').text
publisher=browser.find_element(By.CLASS_NAME,'gd_pub').text
publishing =browser.find_element(By.CLASS_NAME,'gd_date').text
rating=browser.find_element(By.CLASS_NAME,'yes_b').text
reviews=browser.find_element(By.CLASS_NAME,'txC_blue').text
sales=browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
price=browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
#ranking=browser.find_element(By.CLASS_NAME,'gd_best').text.split(" | ")[0]
full_text=browser.find_element(By.CLASS_NAME,'gd_best').text
parts=full_text.split(" | ")
ranking_part=parts[0]
ranking=''.join(filter(str.isdigit, ranking_part)) #숫자만 추출
print(ranking_part)
#ranking_weeks=browser.find_element(By.CLASS_NAME,'gd_best').text.split(" | ")[1]

IndexError: list index out of range

In [5]:

#데이터 베이스 연동 후 수집한 데이터를 db에 저장
import pymysql
import time
import re
from datetime import datetime
from selenium.webdriver.common.by import By

link_list =[]
for i in range(1,4):
    url=f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)

    datas = browser.find_elements(By.CLASS_NAME,'gd_name')
    for i in datas:
        link_list.append(i.get_attribute("href"))
    time.sleep(3)
print(link_list)
print(len(link_list))

conn =pymysql.connect(
    host='localhost',
    user='root',
    password='0310',
    db='yes24',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)

        title=browser.find_element(By.CLASS_NAME,'gd_name').text
        author=browser.find_element(By.CLASS_NAME,'gd_auth').text
        publisher=browser.find_element(By.CLASS_NAME,'gd_pub').text
        publishing =browser.find_element(By.CLASS_NAME,'gd_date').text

        match=re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        if match:
            year,month,day=match.groups()
            data_obj=datetime(int(year),int(month),int(day))
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing="2023-01-01"

        
        
        rating=(browser.find_element(By.CLASS_NAME,'yes_b').text).replace(",","")
        if rating=='':
            rating=0
        elif float(rating)>10:
            rating=0


        reviews=(browser.find_element(By.CLASS_NAME,'txC_blue').text).replace(",","")
        if reviews=='':
            reviews=0
        elif reviews.isdigit()==False:
            reviews=0

        sales=browser.find_element(By.CLASS_NAME,'gd_sellNum').text.split(" ")[2]
        sales=int(sales.replace(",",""))

        price=browser.find_element(By.CLASS_NAME,'yes_m').text[:-1]
        price=int(price.replace(",",""))

        full_text=browser.find_element(By.CLASS_NAME,'gd_best').text
        parts=full_text.split(" | ")
        if len(parts) == 1:
            ranking=0
            ranking_weeks=0
        
        else :
            try:
                ranking_part=parts[0]
                ranking=''.join(filter(str.isdigit, ranking_part)) #숫자만 추출
            except:
                ranking=0

            try:
                ranking_weeks_part=parts[1]
                ranking_weeks=''.join(filter(str.isdigit, ranking_weeks_part))
            except:
                ranking_weeks=0


        

        sql="""insert into Books(
        title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks    )
        values(
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )"""

    
        cur.execute(sql,(title, author, publisher, publishing, rating, reviews, sales, price, ranking, ranking_weeks))
        conn.commit()
        time.sleep(2)



['https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/126010792', 'https://www.yes24.com/Product/Goods/125992096', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/126029503', 'https://www.yes24.com/Product/Goods/125491624', 'https://www.yes24.com/Product/Goods/125651056', 'https://www.yes24.com/Product/Goods/125970283', 'https://www.yes24.com/Product/Goods/125576919', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/126029190', 'https://www.yes24.com/Product/Goods/125973831', 'https://www.yes24.com/Product/Goods/90926731', 'https://www.yes24.com/Product/Goods/96547408', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/124043812', 'https://www.yes24.com/Product/Goods/123930880', 'https://www.yes24.com/Product/Goods/125384202', 'https://www.yes24.co

In [6]:
url='https://www.yes24.com/Product/Goods/125970283'
browser.get(url)
title=browser.find_element(By.CLASS_NAME,'gd_name').text
print(title)
rating=(browser.find_element(By.CLASS_NAME,'yes_b').text).replace(",","")

print(rating)

if float(rating)>10:
    rating=0
print(rating)

reviews=(browser.find_element(By.CLASS_NAME,'txC_blue').text).replace(",","")
if reviews=='':
    reviews=0
elif reviews.isdigit()==False:
    reviews=0
print(reviews)

사카모토 데이즈 15 더블특전판
5400
0
0


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 사용자 에이전트 설정 (모바일 환경으로 설정)
user_agent = "Mozilla/5.0 (IPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/605.1.15(KHTML, like Gecko) CriOS/80.0.3987.95 Mobile/15E148 Safari/604.1"

# 크롬 옵션 설정
options = Options()
options.add_argument(f"user-agent={user_agent}")
options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 드라이버 설정 및 실행
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 멜론 모바일 페이지 접속
url = "https://m2.melon.com/index.htm"
driver.get(url)
time.sleep(1)  # 페이지 로드 대기

# 이벤트 페이지를 감지하고 넘어가는 함수 정의
def check_and_pass_event_page():
    try:
        event_banner = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'ir-pm'))
        )
        melon_logo = driver.find_element(By.XPATH, '/html/body/div[2]/header/h1/a/img')
        melon_logo.click()
    except:
        pass

check_and_pass_event_page()

# 차트 항목 클릭
chart_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="naviMenu"]/nav/ul/li[3]/a'))
)
chart_link.click()

# 스크롤을 통해 페이지 끝까지 로드하는 함수 정의
def scroll_down_inf():
    prev_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == prev_height:
            break
        prev_height = new_height

scroll_down_inf()

# "더보기" 버튼 클릭
def click_load_more():
    button_selector = "#moreBtn"
    driver.execute_script("hasMore2();")
click_load_more()

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.action_chains import ActionChains


def load_category_page(driver, section_url):
    driver.get(section_url)
    link_elements = driver.find_elements(By.CLASS_NAME, 'Nitem_sub_link')
    return [element.get_attribute("href") for element in link_elements]

def get_article(driver, article_url):
    #load_all_articles(driver, article_url)
    articles= driver.find_elements(By.CLASS_NAME, 'sa_text_title')
    return [element.get_attribute("href") for element in articles]

def get_information(driver, article_url):
    driver.get(article_url)
    url=article_url
    title=driver.find_element(By.ID,'title_area').text
    print(title)

def main():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install(),options=chrome_options))

    section_url = 'https://news.naver.com/section/101'
    article_links = load_category_page(driver, section_url)   
    for link in article_links:
        articles = get_article(driver, link)       
        for article in articles:
            get_information(driver,article)
            break
        break  # 테스트끝나면 삭제


if __name__ == "__main__":
    main()

소액생계비대출 상환땐 횟수 제한없이 재대출
